In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_csv("train.csv")
dig_mnist = pd.read_csv("Dig-MNIST.csv")
test_df = pd.read_csv("test.csv")

In [3]:
train_df = pd.concat([train_df, dig_mnist], axis=0, ignore_index=True)

In [4]:
train_df.shape

(70240, 785)

In [5]:
y_train = train_df["label"].values
y_val = dig_mnist["label"].values

In [6]:
x_train = train_df[train_df.keys().drop(["label"])].values
x_test = test_df[test_df.keys().drop(["id"])].values

In [7]:
x_test.shape

(5000, 784)

In [8]:
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1, 28,28, 1)

In [9]:
x_train = x_train/255
x_test = x_test/255

In [10]:
x_train.shape, x_test.shape, y_train.shape

((70240, 28, 28, 1), (5000, 28, 28, 1), (70240,))

In [11]:
set(y_train), set(y_val)

({0, 1, 2, 3, 4, 5, 6, 7, 8, 9}, {0, 1, 2, 3, 4, 5, 6, 7, 8, 9})

In [12]:
y_train = to_categorical(y_train)

In [13]:
y_train.shape

(70240, 10)

In [14]:
def plot_data(number):
    rows = 2
    columns = 5
    fig, ax = plt.subplots(nrows=rows, ncols=columns+1, figsize=(20,20))
    fig.tight_layout()
    j = 0
    for i in range(number, 10*columns+1, 10):
        ax[0][j].imshow(x_train[i][:,:,0])
        ax[1][j].imshow(x_val[i][:,:,0])
        j+=1

In [15]:
# plot_data(number=0)

In [16]:
def create_model():
    model = Sequential()
    model.add(Conv2D(filters = 16, kernel_size=(3,3), padding="same", activation="relu", input_shape=(28,28,1)))
    model.add(Conv2D(filters = 16, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters = 32, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters = 32, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 64, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters = 64, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters = 128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters = 128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters = 256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters = 256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters = 512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters = 512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(2,2))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(units=256, activation="relu"))
    model.add(Dropout(0.1))
    model.add(Dense(units=64, activation="relu"))
    model.add(Dense(units=10, activation="softmax"))
    return model

In [17]:
model = create_model()
model.compile(optimizer="RMSprop", loss=CategoricalCrossentropy(), metrics=["accuracy", "AUC", "Recall"])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 16)        2320      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
batch_normalization (BatchNo (None, 28, 28, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0

In [19]:
x_train, x_val_, y_train, y_val_ = train_test_split(x_train, y_train, test_size=0.2, random_state=69)

In [20]:
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
)

In [21]:
datagen.fit(x_train)

In [22]:
lr = ReduceLROnPlateau(min_lr=0.00001, patience=4, verbose=1, monitor="val_loss", mode="min")
es = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

In [23]:
BATCH_SIZE=64

In [24]:
history = model.fit(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE), epochs=40, 
                    steps_per_epoch = x_train.shape[0]//BATCH_SIZE, validation_data = (x_val_, y_val_),
                    validation_steps=x_val_.shape[0]//BATCH_SIZE,
                    callbacks=[lr, es], verbose = 1)

  ...
    to  
  ['...']
Train for 878 steps, validate on 14048 samples
Epoch 1/40
878/878 [==============================] - 35s 40ms/step - loss: 0.4262 - accuracy: 0.8670 - AUC: 0.9866 - Recall: 0.8412 - val_loss: 0.1577 - val_accuracy: 0.9566 - val_AUC: 0.9956 - val_Recall: 0.95399 - Recall: 0. - ETA: 45s - loss: 1.3526 - accuracy: 0.5648 - AUC: 0.9017 - Recall:  - ETA: 44s - loss: 1.3194 - accuracy: 0.5750 - AUC: 0.9061 - Recall:  - ETA: 42s - loss: 1.2910 - accuracy: 0.5839 - AUC: 0.9099 - Re - ETA: 40s - loss: 1.2373 - acc - ETA: 34s - loss: 1.0627 - accuracy: 0.6564 - AUC: 0.9373 - Recall: 0.57 - E - - ETA: 14s - loss: 0.6050 - accuracy: 0.8081  - ETA: 13s - loss: 0.5828 - accuracy: 0.8155 -  - ETA: 12s - loss: 0.5640 - accuracy: 0.8217 - AUC: 0.9791 -  - ETA: 11s - loss: 0.5551 - accuracy:  - ETA: 9s - loss: 0 - ETA: 5s - loss: 0.4793 - accuracy: 0.8499 - AUC: 0.983 - ETA: 3s - loss: 0.4644 - accuracy: 0.8549 - AUC: 0.98 - ETA: 2s - loss: 0.4496 - accuracy: 0.8595 - AUC: 0.985

878/878 [==============================] - 38s 43ms/step - loss: 0.0341 - accuracy: 0.9918 - AUC: 0.9991 - Recall: 0.9910 - val_loss: 0.0287 - val_accuracy: 0.9933 - val_AUC: 0.9992 - val_Recall: 0.9928
Epoch 31/40
877/878 [============================>.] - ETA: 0s - loss: 0.0319 - accuracy: 0.9917 - AUC: 0.9993 - Recall: 0.9910
Epoch 00031: ReduceLROnPlateau reducing learning rate to 1e-05.
878/878 [==============================] - 38s 43ms/step - loss: 0.0320 - accuracy: 0.9917 - AUC: 0.9993 - Recall: 0.9910 - val_loss: 0.0308 - val_accuracy: 0.9934 - val_AUC: 0.9992 - val_Recall: 0.9929
Epoch 32/40
878/878 [==============================] - 37s 42ms/step - loss: 0.0340 - accuracy: 0.9916 - AUC: 0.9992 - Recall: 0.9906 - val_loss: 0.0284 - val_accuracy: 0.9932 - val_AUC: 0.9993 - val_Recall: 0.9928
Epoch 33/40
878/878 [==============================] - 36s 41ms/step - loss: 0.0331 - accuracy: 0.9917 - AUC: 0.9992 - Recall: 0.9908 - val_loss: 0.0281 - val_accuracy: 0.9935 - val_AUC: 

In [ ]:
preds = model.predict(x_test)

In [ ]:
preds = np.argmax(preds, axis=1)

In [ ]:
sam = pd.read_csv("sample_submission.csv")

In [ ]:
sam["label"] = preds

In [ ]:
sam.to_csv("submission.csv", index=False)